## Softmax classification
- 클래스가 3개 이상인 경우

## 데이터 로더

In [0]:
import torch
import torch.optim as optim
# For reproducibility
torch.manual_seed(1)

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]    #0,1,2 3개의 클래스 중 하나를 가짐.

x_train = torch.FloatTensor(x_train)  # float
y_train = torch.LongTensor(y_train)   # Long

print(x_train[:5])   # 첫 다섯 개
print(y_train[:5])

# print(x_train.shape)   feature는 4개
# print(y_train.shape)

tensor([[1., 2., 1., 1.],
        [2., 1., 3., 2.],
        [3., 1., 3., 4.],
        [4., 1., 5., 5.],
        [1., 7., 5., 5.]])
tensor([2, 2, 2, 1, 1])
torch.Size([8, 4])
torch.Size([8])


In [0]:
# 아래 모델 돌릴 때 쓰는 함수. 미리 알아보기.

# one hot encoding 하기  [1,0,0]=0, [0,1,0]=1, [0,0,1]=2

print(y_train) # 한 리스트에 각 클래스 값이 존재.
print(y_train.unsqueeze(1)) #각 리스트에 각 클래스 값을 담기

nb_class = 3
nb_data = len(y_train)
# one-hot encoding.
y_one_hot = torch.zeros(nb_data, nb_class)      
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)  # scatter(dim, src, index)
                                   # dim 차원 기준으로 index 위치의 값을 src로 바꾼다.

print(y_one_hot)  # 예측한 클래스를 one-hot encoding으로 바꾼 것
                  # 0=100, 1=010, 2=001

tensor([2, 2, 2, 1, 1, 1, 0, 0])
tensor([[2],
        [2],
        [2],
        [1],
        [1],
        [1],
        [0],
        [0]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## 모델 학습

In [0]:
import torch.nn.functional as F  # for softmax

# 모델 초기화
# feature 4개, 3개의 클래스

nb_class = 3
nb_data = len(y_train) #행의 수

W = torch.zeros((4, nb_class), requires_grad=True)  #nb_class=3. (4개의 featrue,3개의 class)  
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
           # softmax함수를 호출하거나, cross-entropy를 그대로 써도 됌.
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    
    # cost 표현번 1번 예시
      # one-hot encoding.
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
# cost 표현법 2번 예시
# cross_entropy를 사용하면 scatter 함수를 이용한 one_hot_encoding을 안해도 됨
# cost = F.cross_entropy((x_train.matmul(W) + b), y_train)


    # cost로 H(x) 개선 (w 업데이트)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 1.036919
Epoch  200/1000 Cost: 1.010905
Epoch  300/1000 Cost: 0.991541
Epoch  400/1000 Cost: 0.974942
Epoch  500/1000 Cost: 0.959928
Epoch  600/1000 Cost: 0.946149
Epoch  700/1000 Cost: 0.933492
Epoch  800/1000 Cost: 0.921898
Epoch  900/1000 Cost: 0.911302
Epoch 1000/1000 Cost: 0.901625


## 데이터 평가

In [0]:
# 학습된 W,b를 통한 클래스 예측

hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1) 
          # torch.argmax(): hypothesis(=wx)에서 가장 큰 값의 인덱스를 return


print(hypothesis) # 범주형데이터로 바뀌기 전. wx를 구한 상태.
print(predict)    # 예측한 클래스 
print(y_train)    # 실제 데이터.

# 정확도 계산 
correct_prediction = predict.float() == y_train
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

# 하나를 틀렸음. 
# 그래서 정확도는 87.50%

tensor([[0.3633, 0.1321, 0.5046],
        [0.1075, 0.3747, 0.5178],
        [0.0338, 0.4811, 0.4851],
        [0.0120, 0.5703, 0.4177],
        [0.9002, 0.0161, 0.0838],
        [0.1476, 0.7789, 0.0735],
        [0.8375, 0.0747, 0.0878],
        [0.8986, 0.0490, 0.0524]], grad_fn=<SoftmaxBackward>)
tensor([2, 2, 2, 1, 0, 1, 0, 0])
tensor([2, 2, 2, 1, 1, 1, 0, 0])
tensor([ True,  True,  True,  True, False,  True,  True,  True])
The model has an accuracy of 87.50% for the training set.
